In [1]:
from pymongo import MongoClient

In [2]:
client=MongoClient()
client=MongoClient('mongodb://.../')
db=client['eventData']
sen=db.documents_english

In [3]:
from nltk.tokenize import RegexpTokenizer

In [4]:
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

In [7]:
tokenizer = RegexpTokenizer(r'\w+')
# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

In [10]:
%%time
texts = []
actuallyTrained=0;
for i in sen.find():
        try:
            raw = ''.join(i['document']).lower()        
            tokens = tokenizer.tokenize(raw)
            stopped_tokens = [i for i in tokens if not i in en_stop]
            stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
            texts.append(stemmed_tokens) 
            actuallyTrained=actuallyTrained+1
        except:
            pass
print(actuallyTrained)

1161388
CPU times: user 1h 6min 12s, sys: 7min 44s, total: 1h 13min 56s
Wall time: 1h 16min 49s


In [12]:
%%time
dictionary = corpora.Dictionary(texts)

CPU times: user 2min 35s, sys: 1.14 s, total: 2min 36s
Wall time: 2min 37s


In [26]:
dictionary.token2id

{'call': 0,
 '800': 1,
 '342': 2,
 '2507': 3,
 'send': 4,
 'question': 5,
 'websit': 6,
 'extens': 7,
 'umd': 8,
 'edu': 9,
 'hgic': 10,
 'plant': 11,
 'week': 12,
 'giant': 13,
 'pussi': 14,
 'willow': 15,
 'japanes': 16,
 'salix': 17,
 'chaenomeloid': 18,
 's': 19,
 'way': 20,
 'won': 21,
 't': 22,
 'feel': 23,
 'fond': 24,
 'can': 25,
 'pet': 26,
 'seed': 27,
 'catalog': 28,
 'often': 29,
 'safe': 30,
 'pledg': 31,
 'somewher': 32,
 'near': 33,
 'front': 34,
 'assur': 35,
 'custom': 36,
 'compani': 37,
 'carri': 38,
 'genet': 39,
 'engin': 40,
 'univers': 41,
 'maryland': 42,
 'home': 43,
 'garden': 44,
 'inform': 45,
 'center': 46,
 'offer': 47,
 'free': 48,
 'pest': 49,
 'famili': 50,
 'will': 51,
 'also': 52,
 'receiv': 53,
 'friend': 54,
 'tuesday': 55,
 '11': 56,
 '30': 57,
 'march': 58,
 'life': 59,
 'tribut': 60,
 '5616': 61,
 'old': 62,
 'court': 63,
 'road': 64,
 'follow': 65,
 'funer': 66,
 'servic': 67,
 '12': 68,
 'pm': 69,
 'paid': 70,
 'death': 71,
 'notic': 72,
 'view

In [14]:
%%time
corpus = [dictionary.doc2bow(text) for text in texts]

CPU times: user 2min 17s, sys: 34.8 s, total: 2min 52s
Wall time: 2min 58s


In [15]:
%%time
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=20, id2word = dictionary, passes=1)

CPU times: user 43min, sys: 1min 25s, total: 44min 25s
Wall time: 44min 36s


In [110]:
ldamodel[corpus[1]]

[(5, 0.57942845600499893), (8, 0.39703790016898888)]

In [44]:
ldamodel

In [16]:
actuallyTrained

1161388

#change result into multidimensinal array in order to feed in k-means model,
#the no of dimension is the no of topics

In [18]:
%%time
dim=20
result=[]
for i in range(0,actuallyTrained):
    feature=[]
    previousindex=0
    for item in ldamodel[corpus[i]]:
        index=item[0]
        #print(index)
        for beforeindex in range(previousindex,index):
            feature.append(0)
        feature.append(item[1])
        previousindex=index+1
    while (len(feature)<dim):
        feature.append(0);  #add in 0 at the end
    result.append(feature)

CPU times: user 23min 14s, sys: 1min 38s, total: 24min 52s
Wall time: 3h 15min 36s


In [20]:
from sklearn.cluster import KMeans
import numpy as np

In [21]:
%%time
kmeanstest=np.array(result)

CPU times: user 2.03 s, sys: 193 ms, total: 2.22 s
Wall time: 2.23 s


In [77]:
X = np.array([[1, 2,3], [1, 4,3], [1, 0,3],
...               [4, 2,4], [4, 4,5], [4, 0,6]])

In [23]:
%%time
kmeans = KMeans(n_clusters=20, random_state=0).fit(kmeanstest)

CPU times: user 2min 47s, sys: 1min 5s, total: 3min 52s
Wall time: 3min 41s


In [28]:
kmeans.labels_

array([ 5, 19, 19, ...,  1,  2,  9], dtype=int32)